<a href="https://colab.research.google.com/github/lukaszplust/Projects/blob/main/moje_sbd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import itertools
import math
from random import randint
import pdb

In [48]:
# BUFFER_SIZE - określa liczbę rekordów, które będą przechowywane w buforze w jednym momencie.
# Buforowanie poprawia wydajność operacji wejścia-wyjścia, minimalizując liczbę operacji odczytu i zapisu na dysk poprzez grupowanie ich w większe bloki.

# zakładam, ze blok to 512 bajtów, wiec wychodzi po 32 rekordy
BUFFER_SIZE = 32
#dlaczego rekord to 16 bitów?
# to chyba dlatego co jest nizej

# TO NADANE PRZEZ POLECENIE
# SET_BYTES_SIZE - określa rozmiar w bajtach pojedynczego rekordu bez dodatkowych znaków.
SET_BYTES_SIZE = 15

# Ta stała określa rozmiar w bajtach pojedynczego rekordu, włączając dodatkowy znak (nowa linia lub znak końca rekordu).
# Zazwyczaj jest to znak '\n' (nowa linia), który oddziela rekordy w pliku tekstowym
RECORD_BYTES_SIZE = SET_BYTES_SIZE + 1

# Ta stała określa całkowity rozmiar bufora w bajtach.
# Jest to iloczyn liczby rekordów w buforze (BUFFER_SIZE) i rozmiaru jednego rekordu w bajtach (RECORD_BYTES_SIZE).
# Bufor o takim rozmiarze będzie używany do operacji odczytu i zapisu blokowego, co zwiększa efektywność przez minimalizację liczby operacji I/O
BYTES_BUFFER_SIZE = BUFFER_SIZE * RECORD_BYTES_SIZE

In [1]:
class Record:

  def __init__(self, elements):
    self.elements = elements

  def __repr__(self):
    return f"{sorted(self.elements, reverse=True)}"

  #Przykład: ints = [3, 10, 20, 30], to:

  #ints[0] wynosi 3, co oznacza, że są 3 elementy.
  #ints[1: 4] zwraca [10, 20, 30].
  #load_from_ints([3, 10, 20, 30]) zwróci obiekt Record z elementami [10, 20, 30].

  # tworze obiekt Record z listy liczb całkowitych,
  # gdzie pierwsza liczba określa liczbę elementów w Record, a pozostałe liczby są elementami

  # (z wykładu) Zazwyczaj rekordy mają nagłówki pokazujące strukturę rekordu
  @staticmethod
  def load_from_ints(ints):
    return Record(ints[1: ints[0] + 1])

  # przyklad: obiekt Record z elementami [10, 20] i RECORD_BYTES_SIZE wynosi 10:
  #len(self.elements) wynosi 2.
  #self.elements to [10, 20].
  #[0] * (10 - 2 - 1) daje [0] * 7, czyli siedem zer.
  #save_to_ints zwróci [2, 10, 20, 0, 0, 0, 0, 0, 0, 0].

  # konwertuje obiekt Record na listę liczb całkowitych, gdzie pierwszy element to liczba elementów,
  # a reszta to elementy, z dodatkowymi zerami na końcu do osiągnięcia określonego rozmiaru
  def save_to_ints(self):
    return [len(self.elements)] + self.elements + [0] * (RECORD_BYTES_SIZE - len(self.elements) - 1)

  # Metoda __lt__ porównuje dwa obiekty na podstawie ich unikalnych elementów
  def __lt__(self, other):
    if other is None:
        return True

    self_copy = set(self.elements)
    other_copy = set(other.elements)

    unique_self = self_copy - other_copy
    unique_other = other_copy - self_copy

    #sprawdzam czy unique_other jest pusty
    # jesli jest pusty oznacza to, że wszystkie elementy w other są obecne w self
    if not unique_other:
      # takim przypadku self nie jest mniejszy od other, więc metoda zwraca False
      return False

    # #sprawdzam czy unique_self jest pusty
    # jeśli unique_self jest pusty, oznacza to, że wszystkie elementy w self są obecne w other
    if not unique_self:
      # W takim przypadku self jest uznawany za mniejszy od other, więc metoda zwraca True
      return True
    #  Jeśli największy unikalny element w unique_other jest większy niż największy
    # unikalny element w unique_self, metoda zwraca True, co oznacza, że self jest mniejszy od other
    return max(unique_other) > max(unique_self)

In [50]:
class ReadBuffer:

  def __init__(self, path):

    self.path = path
    self.read_possition = 0
    self.file_possition = 0
    self.size = BUFFER_SIZE
    self.file_size = os.path.getsize(path)
    # liczba rekordów obecnie załadowanych do bufora
    self.loaded_size = 0
    self.buffer = []
    self.disk_reads_count = 0
    #ładowanie początkowego zestawu danych do bufora
    self.load_next()

  def check_more(self):
    return (self.file_possition < self.file_size or self.read_possition < self.loaded_size)

  # see_next uzywać w SeriesIterator, gdzie weryfikuje gdzie konczy sie seria
  # (porownuje kolejne liczba z ta z poprzednia i ustawiam flage end_of_series)
  # dzieki see_next() mogą podglądnąc kolejny record bez przesuwania wskaznika

  def see_next(self):
    return None if self.read_possition == self.loaded_size else self.buffer[self.read_possition]

  # ODCZYT REKORDÓW: Metoda read_next zwraca następny rekord
  def read_next(self):
    # sprawdzam, czy są jeszcze rekordy do odczytu
    if not self.check_more():
        return None

    # jeśli są jakieś rekordy do odczytu to zwracam następny rekord z bufora
    record = self.buffer[self.read_possition]
    self.read_possition += 1

    # jeśli pozycja odczytu osiągnie rozmiar bufora
    if self.read_possition == self.size:
        # ładuje kolejną porcję danych do bufora (self.load_next())
        self.load_next()
        # resetuje self.read_possition
        self.read_possition = 0

    return record

  def load_next(self):
    self.buffer = []
    with open(self.path, "rb", buffering =0) as f:
      # seek() function is used to change the position of
      # the File Handle to a given specific position
      # przesuwam wskaźnik odczytu pliku do self.file_pos
      f.seek(self.file_possition)

      bytes_to_read = min(BYTES_BUFFER_SIZE, self.file_size - self.file_possition)

      temporrary_buffer = f.read(bytes_to_read)

      self.file_possition += bytes_to_read
      self.loaded_size = bytes_to_read / RECORD_BYTES_SIZE
      temporrary_ints = list(temporrary_buffer)

      # dodaje liste rekordów do bufora (self.buffer)
      for i in range(len(temporrary_buffer) // RECORD_BYTES_SIZE):
          record_ints = temporrary_ints[
                        RECORD_BYTES_SIZE * i:RECORD_BYTES_SIZE * (i + 1)
                        ]
          self.buffer.append(Record.load_from_ints(record_ints))

      # zamykam plik i zwiększam licznik operacji odczytu z dysku (self.disk_reads_count)
      f.close()
      self.disk_reads_count += 1


  def __iter__(self):
    return self

  def __next__(self):
    next_record = self.read_next()
    if next_record is None:
      raise StopIteration
    return next_record


In [51]:
class WriteBuffer:

  def __init__(self, path, append_mode=False):

    self.write_possition = 0
    self.buffer = [None] * BUFFER_SIZE

    self.path = path
    self.size = BUFFER_SIZE

    self.series_written = 0
    self.last_written = None
    self.disk_writes_count = 0

    if not append_mode and os.path.isfile(path):
      os.remove(path)


  def save_next(self):
    ints_to_write = []

    # dla każdego rekordu wywołuje metodę save_to_ints, która konwertuje rekord na listę liczb całkowitych
    for record in self.buffer[0:self.write_possition]:
      ints_to_write += record.save_to_ints()
    # otwieram plik w trybie append binary ("ab"), co oznacza, że dane będą dodawane na końcu pliku w trybie binarnym
    # ustawienie buffering=0 oznacza, że operacje zapisu będą natychmiastowe, bez buforowania
    with open(self.path,"ab", buffering = 0) as f:
      f.write(bytearray(ints_to_write))
      f.close()
      self.disk_writes_count += 1


  def flush(self):
    # setattr ustawia atrybut write_possition obiektu self na 0
    self.write_possition > 0 and (self.save_next() or setattr(self, 'write_possition', 0))

  def write_next(self, record):

    # jeśli bieżący rekord jest mniejszy od ostatnio zapisanegom, to oznacza to początek nowej serii.
    # dlatego licznik zapisanych serii (self.series_written) jest zwiększany o 1
    if record < self.last_written:
      self.series_written += 1
    # jeśli pozycja zapisu osiągnie rozmiar bufora, oznacza to, że bufor jest pełny i trzeba go opróżnić
    if self.write_possition == self.size:
      self.flush()
    # zapis bieżącego rekordu na bieżącej pozycji w buforze
    self.buffer[self.write_possition] = record
    # zwiększam aktualną pozycję zapisu o 1, przygotowując sie do zapisu następnego rekordu
    self.write_possition += 1
    # ustawiam ostatnio zapisany rekord na bieżący rekord, aby można było go porównać z następnym rekordem
    self.last_written = record

In [52]:
# Dzięki klasie SeriesIterator weryfikuje gdzie konczy się seria, wiec mogę otrzymać rekordy podzielone na serie
class SeriesIterator:

  def __init__(self, read_buffer):
    self.read_buffer = read_buffer
    self.end_of_series = False
    self.current_record = None



  def read_next(self):
    #pdb.set_trace()

    # jesli zosała ustawiona flaga konca serii to zwracam None
    if self.end_of_series:
      return None

    # odczytuje kolejne rekordy i przypisuje je do current_record
    # tu nalezy zaznaczyc ze read_next() sam przesunie wskaznik na
    # kolejny rekord
    self.current_record = self.read_buffer.read_next()

    if self.current_record is None:
      return None

    # podgladam jaki bedzie nastepny rekord
    next_record = self.read_buffer.see_next()

    # tu jesli pogdlądniety rekord jest mniejszy od poprzedniego to oznacza ze zakonczyła sie pewna seria
    if next_record is not None and next_record < self.current_record:
      # i wtedy nalezy ustawic flage konca serii na true
      self.end_of_series = True

    # zwracam obecny rekord
    return self.current_record

  def __iter__(self):
    return self

  def __next__(self):
    # jesli została ustawiona flaga en_of_series na True to self.read_next() będzie równy None
    # i wtedy jest StopIteration bo koniec serii
    res_record = self.read_next()
    if res_record is None:
      raise StopIteration
    return res_record

In [53]:
def print_tape(name):

  buffer = ReadBuffer(name)
  series_counter = 0
  records_counter = 0
  while buffer.check_more():
    #pdb.set_trace()
    iterator = SeriesIterator(buffer)
    series = []
    for r in iterator:
      series.append(r)
      records_counter +=1
    # gdy wyjdzie z petli to dodaje '||' na koniec serii
    print(" ".join(map(str, series)), end=" | " if buffer.check_more() else "\n")

    # i zwiekszam licznik serii
    series_counter +=1

  print(f'\nSeries count: {series_counter}')
  print(f'Records count: {records_counter}')

def series_count(name):
  series_counter = 0
  buffer = ReadBuffer(name)
  # dopoki bufor zawiera jakies wartosci
  while buffer.check_more():
    #zliczam serie dzieki SeriesIterator, który weryfikuje gdzie konczy sie seria
    iterator = SeriesIterator(buffer)
    for _ in iterator:
      pass
    series_counter +=1
  return series_counter

In [54]:
#Funckja split rozdziela rekordy z pliku źródłowego na dwa docelowe pliki, tworząc w nich sekwencje posortowane
def split(first_tape, second_tape,tape):

  t1_buffer = ReadBuffer(tape)
  t2_buffer = WriteBuffer(first_tape)
  t3_buffer = WriteBuffer(second_tape)

  # następny rekord z pliku źródłowego za pośrednictwem bufora odczytu (ReadBuffer)
  last_record = t1_buffer.read_next()
  # do bufora t2 zapisuje 1 element z t1
  t2_buffer.write_next(last_record)

  destination_buffer = t2_buffer
  # zaczynam zapisywac do t2 i jesli kolejny record jest wiekszy od poprzednika to zapisuje w t3
  # calosc opiera sie na odpowiedniej zmianie destination_buffer
  for r in t1_buffer:
    #pdb.set_trace()
    if r < last_record:
      destination_buffer = t3_buffer if destination_buffer == t2_buffer else t2_buffer

    (t2_buffer if destination_buffer == t2_buffer else t3_buffer).write_next(r)
    last_record = r

  # Flushing buforów na końcu zapisuje wszystkie pozostałe dane z buforów do plików, zapewniając,
  # że wszystkie dane są zapisywane i bufor jest pusty przed kolejnymi operacjami
  t2_buffer.flush()
  t3_buffer.flush()

  #dodane

  print("\nTape 1: ")
  #print(t2_buffer)
  print_tape(first_tape)
  print("\nTape 2 :")
  print_tape(second_tape)

  return Metadane(t1_buffer.disk_reads_count,
  # t2_buffer.disk_writes_count + t3_buffer.disk_writes_count: Łączna liczba operacji zapisu na dysku dla obu buforów
  t2_buffer.disk_writes_count + t3_buffer.disk_writes_count,
  # t2_buffer.series_written + t3_buffer.series_written: Łączna liczba sekwencji posortowanych (runs) zapisanych do obu buforów
  t2_buffer.series_written + t3_buffer.series_written)

In [55]:
# Funkcja series_merge jest używana do scalenia dwóch posortowanych sekwencji rekordów w jedną posortowaną sekwencję
def series_merge(first_tape, second_tape, write_buffer: WriteBuffer):

  # current_ser1 - kolejne wartosc z t1
  records_in_tape1 = first_tape.read_next()
  # current_ser2 - kolejne wartosc z t2
  records_in_tape2 = second_tape.read_next()
  #pdb.set_trace()
  while records_in_tape1 is not None and records_in_tape2 is not None:

    # sprawdzam czy rekord z tasmy 1 jest mniejszy niz z tasmy 2
    if records_in_tape1 < records_in_tape2:
      # jesli rekord z tasmy 1 jest mniejszy to zapisuje go (ten z pierwszej tasmy) do buffora
      write_buffer.write_next(records_in_tape1)
      # i odczytuje kolejny rekord
      records_in_tape1 =  first_tape.read_next()
    else:
      # jesli rekord z tasmy 1 jest wiekszy niz z tasmy 2 to zapisuje rekord (z drugiej tasmy) do bufora
      write_buffer.write_next(records_in_tape2)
      # odczytuje kolejny rekord z drugiej tasmy
      records_in_tape2 = second_tape.read_next()

  # gdy wyjde poza tasme
  for tape in (first_tape, second_tape):
    #pdb.set_trace()
    # teraz sprawdzam czy to z tasmy 1 czy z tasmy 2 wyszedłem poza nią (czyli uzyskałem None)
    possibly_none_value = records_in_tape1 if tape == first_tape else records_in_tape2
    #pdb.set_trace()
    if possibly_none_value is not None:
      # brakujacy rekord z taśmy z której nie wyszedłem zapisuje do write_buffer
      write_buffer.write_next(possibly_none_value)
      # na taśmie z której nie uzyskałem None pozostały mi jeszcze wartości


      # musze jeszcze sprawdzic pozostałe rekordy z tej
      # z ktorej jeszcze nie wypisałem wszystkich rekordow
      for remaining_records in tape:
        #pdb.set_trace()
        #print(f"Brakujące: {remaining_records}")
        #zapisuje pozstałe rekordy
        write_buffer.write_next(remaining_records)

In [56]:
# Funkcja merge łączy dwie posortowane tasmy (tasme1 i tasme2) w jedną większą posortowaną sekwencję (wywołując series_merge)
def merge(first_tape, second_tape, tape):
  t1_buffer = WriteBuffer(tape)

  t2_buffer = ReadBuffer(first_tape)# tasma 1
  t3_buffer = ReadBuffer(second_tape)# tasma 2
  #pdb.set_trace()
  while t2_buffer.check_more() and t3_buffer.check_more():
    series_merge(SeriesIterator(t2_buffer), SeriesIterator(t3_buffer), t1_buffer)

  # z bufora2 i bufora3 zapisuje wszystko do bufora 1
  for buffer in (t2_buffer, t3_buffer):
    for r in buffer:
        t1_buffer.write_next(r)

  t1_buffer.flush()

  return Metadane(t2_buffer.disk_reads_count + t3_buffer.disk_reads_count,
                  t1_buffer.disk_writes_count,
                  t1_buffer.series_written)

In [57]:
def prepare_tapes():

  tape1 = WriteBuffer("tapes/t1")

  # kopuje wszystko z wejsciowej tasmy na tasme1
  for r in ReadBuffer("tapes/start_tape"):
    tape1.write_next(r)

  tape1.flush()

  for tape in ("t2", "t3"):
    path = f"tapes/{tape}"
    if os.path.isfile(path):
        os.remove(path)

In [58]:
def tape_sort(tape):

  phases_count = 0
  series_written = 0

  reads_count = 0
  writes_count = 0
  # jeśli liczba serii wynosi 1, wszystkie dane na taśmie są już posortowane
  while series_written != 1:

    split_information = split("tapes/t2", "tapes/t3", tape)
    merge_information = merge("tapes/t2", "tapes/t3", tape)

    series_written = merge_information.series_count

    reads_count += split_information.reads_count
    reads_count += merge_information.reads_count

    writes_count += split_information.writes_count
    writes_count += merge_information.writes_count


    print(f'\nPhase: {phases_count + 1}')
    print("Tape 3:")
    print_tape(tape)

    phases_count +=1

  # zwracam obiekt MetaSorting zawierający liczbę odczytów, zapisów i faz sortowania, co pozwala na analizę wydajności algorytmu.
  return MetaSorting(reads_count, writes_count, phases_count)


In [59]:
from collections import namedtuple

Metadane = namedtuple('Metadane', ['reads_count', 'writes_count', 'series_count'])
MetaSorting = namedtuple('MetaSorting', ['reads_count', 'writes_count', 'phases_count'])

Wywoływanie

In [60]:
folder_name = 'tapes'
os.makedirs(folder_name, exist_ok=True)

In [61]:
!rm tapes/t1
!rm tapes/t2
!rm tapes/t3

In [62]:
tape = "tapes/t1"
test_file_path = "tapes/test"

In [63]:
write_buffer = WriteBuffer(tape, append_mode=True)
count = 0

with open(test_file_path) as test_file:
  for line in test_file:
    set_numbers = [int(s) for s in line.rstrip().split()]
    new_record = Record(set_numbers)
    write_buffer.write_next(new_record)
    count += 1
write_buffer.flush()
print(f'Added {count} records to tape')

Added 8 records to tape


In [64]:
#print(f"Displaying tape {tape}")
#print_tape(tape)

In [65]:
print(f"Sorting tape {tape}")
print(f"Tape before sorting:")
print_tape(tape)
sort_info = tape_sort(tape)
#sort_info = tape_sort(tape)
print(f"\nTape after sorting:")
print_tape(tape)
print(f"Metadata:")
print(f"Phase counter: {sort_info.phases_count}")
print(f"Reads counter: {sort_info.reads_count}")
print(f"Writes counter: {sort_info.writes_count}")

Sorting tape tapes/t1
Tape before sorting:
[44] [55] | [12] [42] [94] | [18] | [6] [67]

Series count: 4
Records count: 8

Tape 1: 
[44] [55] | [18]

Series count: 2
Records count: 3

Tape 2 :
[12] [42] [94] | [6] [67]

Series count: 2
Records count: 5

Phase: 1
Tape 3:
[12] [42] [44] [55] [94] | [6] [18] [67]

Series count: 2
Records count: 8

Tape 1: 
[12] [42] [44] [55] [94]

Series count: 1
Records count: 5

Tape 2 :
[6] [18] [67]

Series count: 1
Records count: 3

Phase: 2
Tape 3:
[6] [12] [18] [42] [44] [55] [67] [94]

Series count: 1
Records count: 8

Tape after sorting:
[6] [12] [18] [42] [44] [55] [67] [94]

Series count: 1
Records count: 8
Metadata:
Phase counter: 2
Reads counter: 6
Writes counter: 6
